In [1]:
import os

import pprint as pp
from IPython.display import IFrame

import rangekeeper as rk

In [2]:
# Authenticate with Speckle
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN')) # Note if you run this notebook yourself, you'll need to set this environment variable



 SpeckleClient( server: https://speckle.xyz, authenticated: True )


In [3]:
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)

In [4]:
# Load the design scenario
model = speckle.get_commit(stream_id=stream_id) # Providing no commit_id defaults to the latest commit
model

Base(id: b914ed89cf5040b5d05ff14d22e49c39, speckle_type: Base, totalChildrenCount: 2898)

In [5]:
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)

In [6]:
# Let's identify the root members of the scenario:
roots = model.get_dynamic_member_names()
roots

['@context', '@scenario']

In [7]:
IFrame("https://speckle.xyz/streams/{0}/objects/{1}".format(stream_id, model['@scenario']['id']), width='100%', height=800)

In [8]:
property_name = 'type'
IFrame('https://speckle.xyz/streams/{0}/commits/{1}?filter=%7B%22propertyInfoKey%22%3A%22{2}%22%7D'.format(
    stream_id,
    commit_id,
    property_name), width='100%', height=800)

In [9]:
# Return any Speckle Objects in the '@scenario' trunk that have 'entityId's:
parsed = rk.api.Speckle.parse(base=model['@scenario'])

# (Recursively) Convert the Speckle Objects into Rangekeeper Entities:
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='scenario',
    type='scenario')

# (We can check that it is an Assembly:)
isinstance(scenario, rk.graph.Assembly)

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


True

In [10]:
# Get the "BuildingA" Assembly:
buildingA = [entity for (entityId, entity) in scenario.get_entities().items() if entity['name'] == 'buildingA'][0]
print('buildingA: {0}'.format(buildingA))

buildingA: Assembly: buildingA
Type: building
Members: ['type', 'name']
Entities: [('eb42cd14-73ab-41fc-99c0-c46631ee1208', {'entity': Assembly: buildingA (Type: building)}), ('655b2dfe-5c18-4d68-95f0-f5d27804430d', {'entity': Assembly: buildingAoffice (Type: space)}), ('47dfa8b6-e213-4099-9f5b-834df0ca34b0', {'entity': Assembly: buildingAretail (Type: space)}), ('da796c3e-94b0-4175-8c6f-ac63e5e60714', {'entity': Assembly: buildingAresidential (Type: space)}), ('08e12c72-4450-424f-8ff4-a4da7246ee05', {'entity': Assembly: buildingAretail (Type: space)}), ('634c5fb4-f204-4287-8c80-f23371056f1c', {'entity': Assembly: buildingAparking (Type: space)}), ('abdf16fd-494e-4d7f-a604-134ce1c004ac', {'entity': Entity: buildingAcores (Type: utilities)})]
Relationships: [('eb42cd14-73ab-41fc-99c0-c46631ee1208', '655b2dfe-5c18-4d68-95f0-f5d27804430d', 'spatiallyContains'), ('eb42cd14-73ab-41fc-99c0-c46631ee1208', '47dfa8b6-e213-4099-9f5b-834df0ca34b0', 'spatiallyContains'), ('eb42cd14-73ab-41fc-99c0-

In [11]:
# Get all relatives of BuildingA where BuildingA is the source of
# a 'spatiallyContains' relationship:
buildingA_containment = buildingA.get_relatives(
    outgoing=True,
    relationship_type='spatiallyContains',
    assembly=scenario)
print('buildingA Containment: \n {0}\n'.format(buildingA_containment))

buildingA Containment: 
 [Assembly: buildingAoffice (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAresidential (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAparking (Type: space), Entity: buildingAcores (Type: utilities)]



In [12]:
buildingAresi = [entity for entity in buildingA_containment if entity['name'] == 'buildingAresidential'][0]
print('buildingAresidential: {0}'.format(buildingAresi))

buildingAresidential: Assembly: buildingAresidential
Type: space
Members: ['renderMaterial', 'use', 'type', 'name', '@displayValue']
Entities: [('da796c3e-94b0-4175-8c6f-ac63e5e60714', {'entity': Assembly: buildingAresidential (Type: space)}), ('a987a169-2605-4308-9462-8d22eb6df2be', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('d2db8f8f-36c9-4a92-a1dd-016376fee217', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('b0384123-9bec-4c0e-87f6-32abb6a49fb3', {'entity': Entity: buildingAresidentialFloor1 (Type: floor)}), ('bbd83975-af58-492e-8818-ce1968564a05', {'entity': Entity: buildingAresidentialFloor2 (Type: floor)}), ('c78fd645-372b-4a23-a823-98e2f6885b97', {'entity': Entity: buildingAresidentialFloor3 (Type: floor)})]
Relationships: [('da796c3e-94b0-4175-8c6f-ac63e5e60714', 'a987a169-2605-4308-9462-8d22eb6df2be', 'spatiallyContains'), ('da796c3e-94b0-4175-8c6f-ac63e5e60714', 'd2db8f8f-36c9-4a92-a1dd-016376fee217', 'spatiallyContains'), ('da796c3e-94b

In [13]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html
